In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import TensorBoard

#Carregar o conjunto de dados Fashion MNIST
(X_treino, y_treino), (X_teste, y_teste) = fashion_mnist.load_data()

#Configurar o TensorBoard
logdir = 'logs/scalars/' + datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = TensorBoard(log_dir=logdir)

#Exibir exemplos de imagens do conjunto de treinamento
for x in range(6):
    plt.subplot(330 + 1 + x)
    plt.imshow(X_treino[x], cmap=plt.get_cmap('gray'))
    plt.show()

#Pré-processar os dados
X_treino = X_treino.astype('float32') / 255
X_teste = X_teste.astype('float32') / 255

X_treino = np.expand_dims(X_treino, -1)
X_teste = np.expand_dims(X_teste, -1)

#Imprimir as dimensões dos dados
print('Dimensão X_treino:', X_treino.shape)
print('Amostras de treino:', X_treino.shape[0])
print('Amostras de teste:', X_teste.shape[0])

#Converter os rótulos para categoricais
num_classes = 10
y_treino = to_categorical(y_treino, num_classes)
y_teste = to_categorical(y_teste, num_classes)

#Criar o modelo
dimensoes_entrada = (28, 28, 1)
modelo = Sequential([
Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=dimensoes_entrada),
MaxPooling2D(pool_size=(2, 2)),
Conv2D(64, kernel_size=(3, 3), activation="relu"),
MaxPooling2D(pool_size=(2, 2)),
Flatten(),
Dropout(0.5),
Dense(num_classes, activation="softmax")
])

print(modelo.summary())

#Compilar e treinar o modelo
tamanho_lote = 128
epocas = 5
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
historico_treinamento = modelo.fit(X_treino, y_treino, batch_size=tamanho_lote, epochs=epocas,
validation_split=0.1, callbacks=[tensorboard_callback])

#Avaliar o modelo
resultado = modelo.evaluate(X_teste, y_teste, verbose=0)
print('Erro no teste:', round(resultado[0], 5))
print('Acurácia no teste:', round(resultado[1], 5))